In [1]:
import pymongo

In [3]:
# initiate a MongoClient instance 
from pymongo import MongoClient
client=MongoClient(host='localhost', port=27017) # can specify host, and port these are default
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
db=client.rptutorials #name of database
# db=client['rptutorials'] # for dictionary like access 
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'rptutorials')

In [5]:
tutorial1={
'title':'Test Title',
'author':'Franz',
'contributors':['p1','p2','p3'],
'url':'fakeurl.com'}

In [6]:
tutorial = db.tutorial # create tutorial collection
tutorial

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'rptutorials'), 'tutorial')

In [7]:
# insert tutorial 1
result = tutorial.insert_one(tutorial1)
result

In [8]:
print(f"One tutorial: {result.inserted_id}")

One tutorial: 62ab7740e2af765ce3dc8294


In [9]:
# for inserting many documents pass a list, where each item is a dictionary
# new_result = tutorial.insert_many([item1, item2])

In [11]:
import pprint
for doc in tutorial.find():
    pprint.pprint(doc)

{'_id': ObjectId('62ab7443fac69062ff65b738'),
 'author': 'Franz',
 'contributors': ['p1', 'p2', 'p3'],
 'title': 'Test Title',
 'url': 'fakeurl.com'}
{'_id': ObjectId('62ab7740e2af765ce3dc8294'),
 'author': 'Franz',
 'contributors': ['p1', 'p2', 'p3'],
 'title': 'Test Title',
 'url': 'fakeurl.com'}


In [12]:
franz_tutorial = tutorial.find_one({'author':'Franz'})

In [15]:
franz_tutorial

{'_id': ObjectId('62ab7443fac69062ff65b738'),
 'title': 'Test Title',
 'author': 'Franz',
 'contributors': ['p1', 'p2', 'p3'],
 'url': 'fakeurl.com'}

In [14]:
pprint.pprint(franz_tutorial) #pprint just prints it better 

{'_id': ObjectId('62ab7443fac69062ff65b738'),
 'author': 'Franz',
 'contributors': ['p1', 'p2', 'p3'],
 'title': 'Test Title',
 'url': 'fakeurl.com'}


In [16]:
# can replace, update, delete documents
# cloosing connections - done when exiting the application to clear all the acquired resources

client.close()

In [ ]:
# to only get what you want and close right after, you can use with statement - context manager protocol
# this automatically calls .close()
# with MongoClient() as client:
#      db = client.rptutorials
#      for doc in db.tutorial.find():
#          pprint.pprint(doc)

#### WORKING WITH MONGOENGINE
* higher abstraction with object-document mapper (similar to SQL object-relational mapper)

In [17]:
from mongoengine import connect
connect(db='rptutorials', host='localhost', port=27017)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [18]:
from mongoengine import Document, ListField, StringField, URLField


In [19]:
# creating a subclass document and provide fields as class attributes, defining a document schema
# similar to making a table in SQL
class Tutorial(Document):
    title = StringField(required=True, max_length=70)
    author = StringField(required=True, max_length=20)
    contributors = ListField(StringField(max_length=20))
    url = URLField(required=True)

In [20]:
# ways to validate fields, and maintain some structure, avoid processing errors and inconsistent behaviour
# db_field - different field name
# required - ensure field is provided 
# default - default value given if none
# unique - no same values on the field

In [21]:
# .save for saving a document to a database 
# note the call to class Tutorial to add 
tutorial1 = Tutorial(
    title="Beautiful Soup: Build a Web Scraper With Python",
    author="Martin",
    contributors=["Aldren", "Geir Arne", "Jaya", "Joanna", "Mike"],
    url="https://realpython.com/beautiful-soup-web-scraper-python/"
)

tutorial1.save()  # Insert the new tutorial

<Tutorial: Tutorial object>

In [23]:
# if a field is missing e.g. title before save - will give an error - as pymongo does datavalidation
# printing specific fields, can also filter the objects printed 
for doc in Tutorial.objects(author='Franz'):
     print(doc.title)

Test Title
Test Title
